In [1]:
import transformers
import pandas as pd
import numpy as np
import nltk
from utils import TweetsDataset, Results
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from collections import defaultdict
import numpy as np
import torch
import sys

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Carga del tokenizador y el modelo preentrenado

In [1]:
tokenizer = transformers.AutoTokenizer.from_pretrained("Newtral/xlm-r-finetuned-toxic-political-tweets-es")
model = transformers.XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=4, problem_type="multi_label_classification")

NameError: name 'transformers' is not defined

## Conjunto de datos 

In [9]:
df = pd.read_csv("../../data/procesed.csv")
df_train, df_valid = TweetsDataset.split_test_val(df,test_size=0.20)
train_data_loader = TweetsDataset.create_data_loader(df_train, tokenizer, batch_size=8)
valid_data_loader = TweetsDataset.create_data_loader(df_valid, tokenizer, batch_size=8)

## Parámetros del entrenamiento

In [10]:
model = model.to(device)
EPOCHS = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_data_loader) * EPOCHS

scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0, 
    num_training_steps = total_steps
)

## Entrenamiento del modelo

In [11]:
import torch.nn.functional as F
import sys

def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    for i,d in enumerate(data_loader):
        print("Entrenando " + "."*(i%4), end="\r")
        loss, logits = model(
            input_ids=d["input_ids"].to(device),
            attention_mask=d["attention_mask"].to(device),
            labels=F.one_hot(d['labels'].to(device), num_classes=4).float(),
            return_dict=False
        )
        logits = logits.detach().cpu().numpy()
        label_ids = d['labels'].cpu().flatten().numpy()
        preds = np.argmax(logits, axis=1).flatten()
        targ = d['labels'].numpy()
        correct_predictions += np.sum(preds==targ)
        losses.append(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        sys.stdout.write("\033[K")
    return correct_predictions / n_examples, np.mean(losses)

def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for d in data_loader:
            loss, logits = model(
                input_ids = d["input_ids"].to(device),
                attention_mask = d["attention_mask"].to(device),
                labels = F.one_hot(d['labels'].to(device), num_classes=4).float(),
                return_dict=False
            )
            logits = logits.detach().cpu().numpy()
            labels_ids = d['labels'].cpu().flatten().numpy()
            preds = np.argmax(logits, axis=1).flatten()
            targ = d['labels'].numpy()
            correct_predictions += np.sum(preds==targ)
            losses.append(loss.item())
        return correct_predictions / n_examples, np.mean(losses)

Epoch 1/3
----------
Train loss: 0.54 Accuracy: 0.345
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 2/3
----------
Train loss: 0.537 Accuracy: 0.352
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 3/3
----------
Train loss: 0.536 Accuracy: 0.358
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364



defaultdict(list,
            {'train_acc': [0.34509803921568627,
              0.35171568627450983,
              0.3575980392156863],
             'train_loss': [0.5398042104407853,
              0.5369225340341431,
              0.5360109250140346],
             'val_acc': [0.3638888888888889,
              0.3638888888888889,
              0.3638888888888889],
             'val_loss': [0.5351561899538393,
              0.5350650320450465,
              0.5347867805648733],
             'f_score': [0.3638888888888888,
              0.3638888888888888,
              0.3638888888888888]})

In [ ]:
from collections import defaultdict
history = defaultdict(list)
best_acc = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-'*10)   
    train_acc, train_loss = train_epoch(model, train_data_loader, optimizer, device, scheduler, len(df_train))
    print(f'Train loss {train_loss} Accuracy {train_acc}')
    val_acc, val_loss = eval_model(model, valid_data_loader, device, len(df_valid))
    print(f'Val loss {val_loss} Accuracy {val_acc}')
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    if val_acc > best_acc:
#         torch.save(model.state_dict(), 'best_model_state_a5.bin')
        best_acc = val_acc

In [ ]:
Results.plot_train_history(history, 'Xml-Newtral')

In [12]:
torch.save(model.state_dict(), 'xlm_newtral_pretrained.bin')

## Prueba del modelo

In [4]:
# Test
from utils import Results, Model
model = transformers.XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=4, problem_type="multi_label_classification")
model.to(device)
model.load_state_dict(torch.load('xlm_newtral_pretrained.bin'))
df_test = pd.read_csv('../../data/development_test.csv')
df_test = pd.DataFrame({
    "tweets": df_test["tweet"],
    "labels": df_test["ideology_multiclass"].map({'moderate_left': 0, 'moderate_right': 1, "left": 2, "right": 3})
})

test_data_loader = TweetsDataset.create_data_loader(df_test, tokenizer)
acc, loss, f1 = Model.test(model, test_data_loader, device, len(df_test))
Results.add_result("newtral-xml", "ideology-multiclass", acc, loss, f1)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Test loss: 0.532 Accuracy: 0.4
